In [1]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

In [2]:
database = DataDB()

In [3]:
ic.LoadInstrumentsDB()

In [4]:
ic.instruments_dict

{'USD_HKD': {'name': 'USD_HKD', 'ins_type': 'CURRENCY', 'displayName': 'USD/HKD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.05, 'displayPrecision': 5},
 'CHF_ZAR': {'name': 'CHF_ZAR', 'ins_type': 'CURRENCY', 'displayName': 'CHF/ZAR', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.05, 'displayPrecision': 5},
 'EUR_ZAR': {'name': 'EUR_ZAR', 'ins_type': 'CURRENCY', 'displayName': 'EUR/ZAR', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.05, 'displayPrecision': 5},
 'AUD_JPY': {'name': 'AUD_JPY', 'ins_type': 'CURRENCY', 'displayName': 'AUD/JPY', 'pipLocation': 0.01, 'tradeUnitsPrecision': 0, 'marginRate': 0.0333, 'displayPrecision': 3},
 'XAG_EUR': {'name': 'XAG_EUR', 'ins_type': 'METAL', 'displayName': 'Silver/EUR', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.1, 'displayPrecision': 5},
 'AUD_USD': {'name': 'AUD_USD', 'ins_type': 'CURRENCY', 'displayName': 'AUD/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision'

In [5]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == 'CURRENCY' and "GBP" in pair:
        pairs.append(pair)

In [6]:
pairs

['GBP_ZAR',
 'GBP_HKD',
 'GBP_CHF',
 'GBP_AUD',
 'GBP_CAD',
 'GBP_SGD',
 'GBP_USD',
 'GBP_PLN',
 'EUR_GBP',
 'GBP_NZD',
 'GBP_JPY']

In [13]:
data = []
api = OandaApi()
for p in pairs:
    print(p)
    d_temp = api.get_candles_df(pair_name = p, granularity="D", count=100)
    d_temp['gain'] = ((d_temp.mid_c - d_temp.mid_o) / d_temp.mid_c) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

GBP_ZAR
GBP_HKD
GBP_CHF
GBP_AUD
GBP_CAD
GBP_SGD
GBP_USD
GBP_PLN
EUR_GBP
GBP_NZD
GBP_JPY


In [14]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
0,2024-08-26,84714,23.38850,23.54154,23.34226,23.49325,23.3510,23.53500,23.31275,23.4835,23.4260,23.54808,23.34977,23.50300,0.445873,GBP_ZAR
1,2024-08-27,81142,23.50200,23.59858,23.45468,23.51225,23.4535,23.58961,23.43450,23.4795,23.5505,23.60765,23.46100,23.54500,0.043594,GBP_ZAR
2,2024-08-28,82376,23.51600,23.53884,23.29209,23.40561,23.4590,23.52700,23.27637,23.3695,23.5730,23.58150,23.30781,23.44172,-0.471639,GBP_ZAR
3,2024-08-29,96013,23.38725,23.44311,23.20490,23.42100,23.3330,23.43381,23.19700,23.3850,23.4415,23.47420,23.21280,23.45700,0.144101,GBP_ZAR
4,2024-09-01,48588,23.42475,23.55436,23.38080,23.43275,23.3855,23.54531,23.36150,23.3875,23.4640,23.56372,23.38874,23.47800,0.034140,GBP_ZAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,2025-01-07,301483,197.18300,197.55600,195.12800,195.79000,197.0870,197.54100,195.11100,195.7610,197.2790,197.57000,195.14300,195.82000,-0.711477,GBP_JPY
1085,2025-01-08,246668,195.75300,195.82000,193.48100,194.59000,195.6200,195.80400,193.46600,194.5490,195.8860,195.88600,193.49600,194.63200,-0.597667,GBP_JPY
1086,2025-01-09,329125,194.61900,194.77900,192.20700,192.56600,194.4820,194.76400,192.19300,192.4830,194.7560,194.81000,192.22000,192.65000,-1.066128,GBP_JPY
1087,2025-01-12,314286,192.59800,192.76100,190.08000,192.17800,192.3730,192.74700,190.06300,192.1110,192.8230,192.82300,190.09600,192.24600,-0.218547,GBP_JPY


In [15]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [17]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [19]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [20]:
calendar_data_df_uk = calendar_data_df[calendar_data_df.country == 'united kingdom']

,date,country,category,event,symbol,actual,previous,forecast
0,2024-12-05,australia,balance of trade,balance of trade,AUITGSB,A$5.953B,,A$5.5B
1,2024-12-05,canada,balance of trade,balance of trade,CATBTOTB,C$-0.92B,,C$-0.7B
2,2024-12-06,canada,unemployment rate,unemployment rate,CANLXEMR,6.8%,,6.5%
3,2024-12-06,united states,non farm payrolls,non farm payrolls,NFP TCH,227K,,194.0K
4,2024-12-06,united states,unemployment rate,unemployment rate,USURTOT,4.2%,,4.1%
...,...,...,...,...,...,...,...,...
176,2025-01-21,germany,zew economic sentiment index,zew economic sentiment index,GERMANYZEWECOSENIND,,,
177,2025-01-21,canada,inflation rate,inflation rate yoy,CACPIYOY,,,1.8%
178,2025-01-22,japan,balance of trade,balance of trade,JNTBAL,,,¥ 100B
179,2025-01-23,australia,business confidence,nab business confidence,NABSCONF,,,3
